In [1]:
from pyecharts import options as opts
from pyecharts.charts import *
from pyecharts.commons.utils import JsCode
import pyecharts.globals as g
import pandas as pd
import datetime as dt
import numpy as np
from math import sqrt
from scipy import stats

g.WarningType.ShowWarning = False


sys.path.append("/Users/louis/Pycharm/rqalpha/")

from rqalpha.data.bar_dict_price_board import BarDictPriceBoard
from rqalpha.data.base_data_source import BaseDataSource
from rqalpha.data.data_proxy import DataProxy
from rqalpha.environment import Environment


def get_df_index_format(df: pd.DataFrame)->str:
    if df.index.hour.any():
        return '%Y%m%d%H%M'
    else:
        return '%Y%m%d'

def chart_line(df: pd.DataFrame, x_col: str=None, y_cols=[], col_names: dict={}, line_width=1, width="100%", height="500px", title='',
               colors=['red', 'orange', 'yellow', 'green', 'olive', 'blue', 'purple'])->Line:
    """
    线图
    :param colors:
    :param line_width:
    :param col_names:
    :param df:
    :param x_col: x轴数据对应列
    :param y_cols: 多个y对应的列
    :param width: '100%'
    :param height:'600px'
    :param title: ""
    :return:
    """
    if x_col is None:
        if str(df.index.dtype) == 'datetime64[ns]':
            date_format = get_df_index_format(df)
            xaxis_data = df.index.strftime(date_format).values.tolist()
        else:
            xaxis_data = df.index.values.tolist()
    else:
        xaxis_data = __fixdatetime(df, x_col).values.tolist()

    line = Line(init_opts=opts.InitOpts(width=width, height=height)) \
        .set_global_opts(title_opts=opts.TitleOpts(title=title), yaxis_opts=opts.AxisOpts(is_scale=True),
                         datazoom_opts=opts.DataZoomOpts()) \
        .add_xaxis(xaxis_data)
    i = 0
    for col in y_cols:
        if col in col_names:
            title = col_names[col]
        else:
            title = col
        line.add_yaxis(title, df[col].values.tolist(), label_opts=opts.LabelOpts(is_show=False),
                       linestyle_opts=opts.LineStyleOpts(color=colors[i % len(colors)], width=line_width))
        i = i + 1
    return line




def chart_kline(df: pd.DataFrame, opn='open', high='high', low='low', close='close',
                volume='volume', kline_overlaps=[], mas=[5, 10, 21], width="100%", height='800px',
                title="")->Kline:
    """
     K线图
    :param df:
    :param indicator:
    :param kline_overlaps:
    :param opn:
    :param high:"high"
    :param low:"low"
    :param close:"close"
    :param volume:"volume"
    :param mas:[5,10,21]
    :param width: "100%"
    :param height: "600px"
    :param title: ""
    :return:
    """
    tooltip_formatter = JsCode("""
     function(params){
        var text='<span>'+params[0].name+'</span>';
        var s_dict={};
        for(var i in params){
            s_dict[params[i].seriesName]=params[i];
        }
        for (var j in s_dict){
            var s=s_dict[j];
            var data=s.data;
            var names=s.dimensionNames;
            text=text+'<br/><span style="color:lightblue;">'+s.seriesName+'</span>';
            if((!(data instanceof Array))){
                text=text+'<span>'+':'+data+'</span>';
            }else if(data instanceof Array & data.length==2){
                text=text+'<span>'+':'+data[1]+'</span>';
            }else if(data instanceof Array & data.length>2){
                for (var k in names){
                    if(k>0 && data instanceof Array){
                        text=text+'<br/><span>'+names[k]+':'+data[k]+'</span>';
                    }
                }
            } 
        }
        return text;
     }
    """)
    if str(df.index.dtype) == 'datetime64[ns]':
        date_format = get_df_index_format(df)
        xaxis_data = df.index.strftime(date_format).values.tolist()
    else:
        xaxis_data = df.index.values.tolist()

    xaxis_index_zoom = [0, 1]
    kline = Kline().add_xaxis(xaxis_data=xaxis_data) \
        .add_yaxis(
        series_name="K线",
        y_axis=df[[opn, close, low, high]].values.tolist(),
        itemstyle_opts=opts.ItemStyleOpts(
            color="#ef232a",
            color0="#14b143",
            border_color="#ef232a",
            border_color0="#14b143",
        ),
        markpoint_opts=opts.MarkPointOpts(
            label_opts=opts.LabelOpts(position="inside", color="#fff"),
            data=[
                opts.MarkPointItem(type_='max', name="最大值"),
                opts.MarkPointItem(type_='min', name="最小值")
            ]
        ),
    ).set_global_opts(
        title_opts=opts.TitleOpts(title=title, pos_left="0", pos_top="0"),
        legend_opts=opts.LegendOpts(type_='scroll', pos_left='0', pos_top='30'),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            is_scale=True,
            boundary_gap=False,
            axisline_opts=opts.AxisLineOpts(is_on_zero=False),
            splitline_opts=opts.SplitLineOpts(is_show=False),
            split_number=20,
            min_="dataMin",
            max_="dataMax",
            is_show=False
        ),
        yaxis_opts=opts.AxisOpts(is_scale=True, splitline_opts=opts.SplitLineOpts(is_show=True),min_=df[low].min()),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="line", formatter=tooltip_formatter),
        datazoom_opts=[
            opts.DataZoomOpts(is_show=False, type_="inside", xaxis_index=[0, 0], range_end=100),
            opts.DataZoomOpts(is_show=True, xaxis_index=xaxis_index_zoom, pos_top="95%", range_end=100),
        ],
        # 三个图的 axis 连在一块
        axispointer_opts=opts.AxisPointerOpts(
            is_show=True,
            link=[{"xAxisIndex": "all"}],
            label=opts.LabelOpts(background_color="#777"),
        )
    )
    bar_volume = (
        Bar().add_xaxis(xaxis_data=xaxis_data).add_yaxis(
            series_name="成交量",
            y_axis=list((df[volume] / 1000000).round(2).values),
            xaxis_index=0,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False)
        )
    )
    if len(mas) > 0:
        line = Line().add_xaxis(xaxis_data=xaxis_data)
        for ma_len in mas:
            line.add_yaxis(
                series_name="MA" + str(ma_len),
                y_axis=df[close].rolling(ma_len).mean().round(2).values.tolist(),
                linestyle_opts=opts.LineStyleOpts(opacity=0.5, width=2),
                label_opts=opts.LabelOpts(is_show=False)
            )
        # Overlap Kline + Line
        kline = kline.overlap(line)
    if len(kline_overlaps) > 0:
        for chart in kline_overlaps:
            kline = kline.overlap(chart)

    # 最后的 Grid
    grid_chart = Grid(init_opts=opts.InitOpts(width=width, height=height))
    grid_chart.add(
            kline,
            grid_opts=opts.GridOpts(pos_left="5%", pos_right="1%", height="80%"),
            grid_index=0,
        )
    bar_volume.set_global_opts(
        legend_opts=opts.LegendOpts(is_show=False),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value}M")),
        xaxis_opts=opts.AxisOpts(is_show=True, axislabel_opts=opts.LabelOpts(interval=50))
    )
    grid_chart.add(
        bar_volume,
        grid_opts=opts.GridOpts(
            pos_left="5%", pos_right="1%", pos_top="80%", height="18%"),
        grid_index=1
    )
     
    return grid_chart


def chart_kline_overlap_signal(title:str, df_target: pd.DataFrame, df_signal: pd.DataFrame, cols={'x': 'date', 'y': 'price', 'value': 'value'},
                               symbol_index=0)->Line:
    date_format = get_df_index_format(df_target)
    symbols = ['circle', 'rect', 'roundRect', 'triangle', 'diamond', 'pin', 'arrow']
    signal_data = []
    for index, row in df_signal.iterrows():
        x_index=pd.to_datetime(df_target.index[df_target.index.get_loc(pd.to_datetime(row[cols['x']]),method='nearest')]).strftime(date_format)
        signal_data.append({'coord': [x_index, row[cols['y']]],
                            'value': row[cols['value']]})
    line1 = chart_line(df_target.iloc[:2], y_cols=['close'], col_names={'close': title}) \
        .set_series_opts(
        markpoint_opts=opts.MarkPointOpts(data=signal_data, symbol=symbols[symbol_index], symbol_size=32,
                                          label_opts=opts.LabelOpts(position="inside", color="black", font_size=18,
                                                                    rotate=0, distance='30%')))
    return line1


class BuySellPoint:
    def __init__(self,backtest_id):
        env = Environment({})
        env.trading_dt = dt.datetime.now()
        self.data_proxy = DataProxy(BaseDataSource("C:/Users/louis.xu/.rqalpha/bundle", {}), BarDictPriceBoard)
        
    def all_securitys(self):
        """所有交易标的"""
        security_names=list(self.df_order['security_name'].unique())
        print(",".join(security_names))
    
    def plot_iter(self,unit='15m'):
        """依次绘制所有标的"""
        security_name=next(self.security_name_iter,None)
        if security_name is not None:
                return self.plot(security_name,unit)
        else:
            print("所有标志结束,重新循环")
            self.security_name_iter=iter(list(self.df_order['security_name'].unique()))
        
    def plot(self,security_name,unit='15m'):
        """绘制指定标的"""
        order_book_id = self.data_proxy.instruments(security_name).order_book_id
        data = data_proxy.history_bars(ins.order_book_id, dt=date, bar_count=count, frequency='1d',
                                                        field=['datetime', 'open', 'high', 'low', 'close', 'limit_up', 'limit_down',
                                                               'volume', 'total_turnover'])
        
        
        df['day']=pd.to_datetime(df['datetime']).dt.floor('d')
        df['label']=df['price'].astype(str)+","+df['amount'].astype(str)
        df['t']=pd.to_datetime(df['time'])
        display(df)
        # 期货和股票买卖记录区分方式不同
        if df['security_name'].str.contains("期货").any():
            buy=df[(df['side']=='long')].copy()
            sell=df[(df['side']=='short')].copy()
        else:
            buy=df[(df['action']=='open')].copy()
            sell=df[(df['action']=='close')].copy()
        if frequency=='1d':
            start=df['day'].min()-dt.timedelta(days=30)
            end=df['day'].max()+dt.timedelta(days=30)
        else:
            start=df['day'].min()-dt.timedelta(days=5)
            end=df['day'].max()+dt.timedelta(days=5)
        df_price=get_price(df['security'].iloc[0],start_date=start,end_date=end,fq=None,frequency=frequency)
        buy['ft']=df_price.index[df_price.index.get_indexer(buy['t'],method='nearest')]
        sell['ft']=df_price.index[df_price.index.get_indexer(sell['t'],method='nearest')]
        buy=buy.merge(df_price[['close']],how='left',left_on='ft',right_index=True)
        sell=sell.merge(df_price[['close']],how='left',left_on='ft',right_index=True)


        buy_signal=chart_kline_overlap_signal("买入",df_price,buy,cols={'x':'ft','y':'close','value':'label'},symbol_index=3)
        sell_signal=chart_kline_overlap_signal("卖出",df_price,sell,cols={'x':'ft','y':'close','value':'label'},symbol_index=2)
        chart=chart_kline(df_price,kline_overlaps=[buy_signal,sell_signal],mas=[5,10,21],title=security_name)
        return chart.render_notebook()

SyntaxError: invalid character '↑' (U+2191) (<ipython-input-1-aad74b5b0918>, line 220)

In [ ]:
bsp=BuySellPoint("434671a9f4aa642044cb55ab44febdab")
bsp.plot("科华生物",'15m')